#### Don't forget to delete the hdmi_out and hdmi_in when finished

# RGB Filter Example
In this notebook, we will explore the colors that are used to create an image. Although humans are able to see a multitude of colors, our eyes are technically only capable of detecting red, green, and blue. Every other color that we see is a composition of these three primary colors. Black is the absense of any color while white is the combination of all colors.

<img src="data/AdditiveColor.png"/>
*This diagram shows how colors are added together to create new colors.*

This notebook will use a video filter that will allow for the addition and removal of colors from a live video feed. This will help show how colors are formed from the three primary colors: red, green, and blue.

#### 1. Download base overlay to the board
Ensure that the camera is not connected to the board. Run the following script to provide the PYNQ with its base overlay.

In [1]:
from pynq.drivers.video import HDMI
from pynq import Bitstream_Part
from pynq.board import Register
from pynq import Overlay

Overlay("demo.bit").download()

#### 2. Connect camera
Physically connect the camera to the HDMI-in port of the PYNQ. Run the following code to instruct the PYNQ to capture the video from the camera and to begin streaming video to your monitor (connected to the HDMI-out port).

In [2]:
hdmi_in = HDMI('in')
hdmi_out = HDMI('out', frame_list=hdmi_in.frame_list)
hdmi_out.mode(2)
hdmi_out.start()
hdmi_in.start()

#### 3. Program board with RGB Filter
Run the following script to download the RGB Filter to the PYNQ. This will allow us to modify the colors of the video stream.

In [3]:
Bitstream_Part("rgb_p.bit").download()

#### 4. Create a user interface
We will communicate with the filter using a nice user interface. Run the following code to activate that interface.

In [4]:
import ipywidgets as widgets
from ipywidgets import Button, HBox, VBox, Label

words = ['HDMI Reset']
items = [Button(description=w) for w in words]


def on_hdmi_clicked(b):
    hdmi_out.stop()
    hdmi_in.stop()
    hdmi_out.start()
    hdmi_in.start()

R0=Register(0)
R1=Register(1)
R2=Register(2)

R0.write(255)
R1.write(255)
R2.write(255)

R0_s = widgets.IntSlider(
    value=255,
    min=0,
    max=255,
    step=1,
    description='Red:',
    disabled=False,
    continuous_update=True,
    orientation='vertical',
    readout=True,
    readout_format='i',
    slider_color='red'
)
R1_s = widgets.IntSlider(
    value=255,
    min=0,
    max=255,
    step=1,
    description='Green:',
    disabled=False,
    continuous_update=True,
    orientation='vertical',
    readout=True,
    readout_format='i',
    slider_color='green'
)
R2_s = widgets.IntSlider(
    value=255,
    min=0,
    max=255,
    step=1,
    description='Blue:',
    disabled=False,
    continuous_update=True,
    orientation='vertical',
    readout=True,
    readout_format='i',
    slider_color='blue'
)

def update_r0(*args):
    R0.write(R0_s.value)
R0_s.observe(update_r0, 'value')
def update_r1(*args):
    R1.write(R1_s.value)
R1_s.observe(update_r1, 'value')
def update_r2(*args):
    R2.write(R2_s.value)
R2_s.observe(update_r2, 'value')

items[0].on_click(on_hdmi_clicked)

widgets.HBox([VBox([items[0]]),R0_s,R1_s,R2_s])

#### 5. Exploration
Feel free to play with the sliders above. As the slider decreases in value, the color associated with that slider will be removed from the video. Likewise, increasing the slider value will add color back into the image.

Notice that when all sliders are reduced to 0 that the image is black. Now, increase the red slider. The image should only include various shades of red. Add green into the image. The video should now include shades of red and green, but also yellow! This is because yellow is the combination of red and green.

#### 6. Clean up
When you are done playing with the RGB filter, run the following code to stop the video stream

In [5]:
hdmi_out.stop()
hdmi_in.stop()
del hdmi_out
del hdmi_in